<a href="https://colab.research.google.com/github/sherlinvarshitha/Generative-AI/blob/main/2266_W6_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.(1 ponto) Design a multilayer ANN architecture according to the requirements shown below. Train, test, save (.h5) and deploy the model to predict the housing price using Keras deep learning library
2.(1 ponto) Calculate training and testing error metrics
3.(1 ponto) Build the application by loading the saved ANN model. Tabela 1: ANN Architecture Layer Neurons Activation Function

Hidden Layer - 1 15 tanh

Hidden Layer - 2 20 tanh

Hidden Layer - 3 15 tanh

Tabela 2: Training Parameters loss function epochs batch size error metric Optimizer Mean Square Error 100 16 Mean Square Error SGD

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.preprocessing import OneHotEncoder

url = '/content/Housing (2).csv'
df = pd.read_csv(url)

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = X_train.select_dtypes(include=['object']).columns

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features_train = encoder.fit_transform(X_train[categorical_features])
encoded_features_test = encoder.transform(X_test[categorical_features])

encoded_df_train = pd.DataFrame(encoded_features_train, columns=encoder.get_feature_names_out(categorical_features), index=X_train.index)
encoded_df_test = pd.DataFrame(encoded_features_test, columns=encoder.get_feature_names_out(categorical_features), index=X_test.index)

X_train = X_train.drop(categorical_features, axis=1)
X_test = X_test.drop(categorical_features, axis=1)
X_train = pd.concat([X_train, encoded_df_train], axis=1)
X_test = pd.concat([X_test, encoded_df_test], axis=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import MeanSquaredError

model = Sequential()

model.add(Dense(15, input_dim=X_train.shape[1], activation='tanh'))

model.add(Dense(20, activation='tanh'))

model.add(Dense(15, activation='tanh'))

model.add(Dense(1))

model.compile(loss='mean_squared_error',
              optimizer=SGD(),
              metrics=[MeanSquaredError()])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Train the model
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=16,
                    validation_data=(X_test, y_test))

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 9784755486720.0000 - mean_squared_error: 9784755486720.0000 - val_loss: 5561054134272.0000 - val_mean_squared_error: 5561054134272.0000
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3210481238016.0000 - mean_squared_error: 3210481238016.0000 - val_loss: 5063817297920.0000 - val_mean_squared_error: 5063817297920.0000
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3306990338048.0000 - mean_squared_error: 3306990338048.0000 - val_loss: 5098389897216.0000 - val_mean_squared_error: 5098389897216.0000
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3282926829568.0000 - mean_squared_error: 3282926829568.0000 - val_loss: 5058229436416.0000 - val_mean_squared_error: 5058229436416.0000
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3034763493376.0000 - mean_squared_error: 3034763493376.0000 - val_loss: 5084117729280.0000 - val_mean_squared_error: 5084117729280.0000
Epoch 6/100
28/28 ━━━━━━

In [5]:
# Calculate the training error
train_loss, train_mse = model.evaluate(X_train, y_train)
print(f"Training MSE: {train_mse}")



14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3125633875968.0000 - mean_squared_error: 3125633875968.0000 
Training MSE: 3086916517888.0


In [7]:
# Calculate the testing error
test_loss, test_mse = model.evaluate(X_test, y_test)
print(f"Testing MSE: {test_mse}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5249724579840.0000 - mean_squared_error: 5249724579840.0000
Testing MSE: 5112960909312.0


In [8]:
# Save the trained model
model.save('/content/housing_price_model.h5')

from tensorflow.keras.models import load_model

loaded_model = load_model('/content/housing_price_model.h5')

predictions = loaded_model.predict(X_test)

for actual, predicted in zip(y_test[:10], predictions[:10]):
    print(f"Actual: {actual}, Predicted: {predicted[0]}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Actual: 4060000, Predicted: 4765895.0
Actual: 6650000, Predicted: 4765895.0
Actual: 3710000, Predicted: 4765895.0
Actual: 6440000, Predicted: 4765895.0
Actual: 2800000, Predicted: 4765895.0
Actual: 4900000, Predicted: 4765895.0
Actual: 5250000, Predicted: 4765895.0
Actual: 4543000, Predicted: 4765895.0
Actual: 2450000, Predicted: 4765895.0
Actual: 3353000, Predicted: 4765895.0
